1. Download

In [ ]:
#pip install opencv-python matplotlib tensorflow mediapipe scikit-learn

2. Import

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt    # MIGHT REMOVE
import time                             # MIGHT REMOVE
import mediapipe as mp
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # MIGHT REMOVE

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils

3. Functions

In [ ]:
# Mediapipe Detection Function
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Color Conversion BGR to RGB
    image.flags.writeable = False                   # Non-writeable
    results = model.process(image)                  # Make Prediction
    image.flags.writeable = True                    # Writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Color Conversion RGB back to BGR
    return image, results

In [ ]:
# Mediapipe Draw Function
def draw_styled_landmarks(image, results):
    mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,                     # Draw Face
                              mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),   # Face Dot Color
                              mp_draw.DrawingSpec(color=(240,19,255), thickness=1, circle_radius=1),    # Face Line Color
                              )

In [ ]:
# Extract Keypoints Function
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    return pose
    #return np.concatenate([pose, face, leftHand, rightHand])

In [ ]:
colors = [(245,118,16),(117,245,16),(16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

TEST Pose Detection

In [ ]:
cap = cv2.VideoCapture(0)
# Set Mediapipe Model
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        # Read Feed
        ret, frame = cap.read()
        
        # Make Detections
        image, results = mediapipe_detection(frame, pose_model)
        print(results)
        
        # Draw Landmarks
        draw_styled_landmarks(image, results)

        # Show Feed
        cv2.imshow('OpenCV Feed', image)

        # Break Loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # If Broken
    cap.release()
    cv2.destroyAllWindows()

4. Training

In [ ]:
# Path for stored data
DATA_PATH = os.path.join('MP_DATA')

# Actions to detect
actions = np.array(['nothing', 'deadlift-s', 'deadlift-c', 'squat-s', 'squat-c']) # S = Starting Point, C = Contraction Point

sequence_length = 30

In [ ]:
# Create Directory
for action in actions: 
    os.makedirs(os.path.join(DATA_PATH, action))

In [ ]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose_model)
        draw_styled_landmarks(image, results)

        cv2.putText(image, 'Collectiong frames for {}'.format(action), (0,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

        cv2.imshow('Data Capture', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('1'):
            action = 'nothing'
        elif key == ord('2'):
            if action == actions[1]:
                action = actions[2]
            else:
                action = actions[1]
        elif key == ord('3'):
            if action == actions[3]:
                action = actions[4]
            else:
                action = actions[3]
        elif key == ord(' '):
            # Make New Directory
            name = str(len(os.listdir(os.path.join(DATA_PATH, action))))
            os.makedirs(os.path.join(DATA_PATH, action, name))

            for i in range(sequence_length):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, pose_model)
                print(results)
                draw_styled_landmarks(image, results)
                cv2.imshow('Data Capture', image)
                npy_path = os.path.join(DATA_PATH, action, name, str(i))
                keypoints = extract_keypoints(results)
                np.save(npy_path, keypoints)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
no_sequences = 30 # Number of recordings made for each action

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [20]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]

In [21]:
x = np.array(sequences)

In [22]:
y = to_categorical(labels).astype(int)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.05)

In [24]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,99)))    # Not sure 99 is the right amount yet
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) # Look up optimizer

In [ ]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback]) # Be vary of the epochs, potentially stop early

In [50]:
model.save('action.keras')
model.save('action.h5')

In [ ]:
model = tf.keras.models.load_model('action.keras')

5. Test Model

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
# Look in other project when ready to test model

In [26]:
testRes = model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [29]:
testRes

array([[1.04762845e-01, 1.27750899e-09, 8.95237207e-01, 4.66473699e-26,
        2.20132424e-25],
       [2.63328248e-10, 2.12057901e-04, 4.35799105e-11, 1.99474040e-02,
        9.79840577e-01],
       [6.79854957e-12, 4.11219526e-06, 9.14623106e-16, 1.85988825e-02,
        9.81397033e-01],
       [3.98063912e-06, 9.91885424e-01, 6.94969094e-06, 1.18080825e-04,
        7.98549037e-03],
       [2.48699166e-06, 9.63228226e-01, 4.49893832e-06, 4.15873888e-04,
        3.63489017e-02],
       [9.81968105e-01, 3.79278913e-06, 1.80281661e-02, 1.50211532e-17,
        2.64438519e-15],
       [1.38179951e-06, 4.98892386e-11, 4.57618925e-12, 9.99998569e-01,
        2.41469496e-12],
       [3.04819969e-06, 9.72552419e-01, 4.97487554e-06, 2.20455157e-04,
        2.72190236e-02]], dtype=float32)

In [28]:
np.sum(testRes[0])

1.0

In [32]:
actions[np.argmax((testRes[0]))]

'deadlift-c'

In [36]:
actions[np.argmax(y_test[0])]

'deadlift-c'

6. Live Predictions

In [88]:
prediction = ''
sequence = []

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose_model)
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        currentFrame = len(sequence)

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sequence = [] # Might not need as it creates 30 frames delay

        currentConf =  res[np.argmax(res)]
        currentRes = actions[np.argmax(res)]

        if currentConf >= 0.7:
            cv2.putText(image, 'Prediction: {}'.format(currentRes), (0,12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Confidence: {}'.format(currentConf), (0,26),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Frames: {}'.format(currentFrame), (0,40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        else:
            cv2.putText(image, 'Prediction: {}'.format(currentRes), (0,12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(image, 'Confidence: {}'.format(currentConf), (0,26),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(image, 'Frames: {}'.format(currentFrame), (0,40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

        cv2.imshow('Prediction Feed', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [79]:
cap.release()
cv2.destroyAllWindows()

In [54]:
np.expand_dims(sequence, axis=0).shape

(1, 30, 99)

In [71]:
actions[np.argmax(res)]

'deadlift-s'

In [77]:
res[np.argmax(res)]

0.5383517

In [46]:
actions[np.argmax(res[0])]

'nothing'